In [1]:
import pandas as pd
import regex
import numpy as np
import warnings
import random
import json
warnings.filterwarnings("ignore", category=Warning)


In [3]:
df2 = pd.read_csv(r"code/drugs.csv")
df2.sample(n=5)

,itemLabel,altLabel_list
752,moxisylyte,Moxisylyte
2259,mycophenolate mofetil,MMF|2-morpholinoethyl (E)-6-(4-hydroxy-6-metho...
1266,posaconazole,Posaconazole
1585,samarium sm-153 lexidronam pentasodium,cyt-424|samarium (153 sm) lexidronam pentasodi...
1133,tecovirimat,ST-246|Tpoxx


In [4]:
df2['alternatives'] = df2['itemLabel'] + "|" + df2['altLabel_list'] 
df2['alternatives'] = df2['alternatives'].str.lower().str.split('|')
df2

,itemLabel,altLabel_list,alternatives
0,propylthiouracil,2-Mercapto-6-propyl-4-pyrimidone|2-Mercapto-6-...,"[propylthiouracil, 2-mercapto-6-propyl-4-pyrim..."
1,gemfibrozil,"2,2-Dimethyl-5-(2,5-dimethylphenoxy)valeriansa...","[gemfibrozil, 2,2-dimethyl-5-(2,5-dimethylphen..."
2,isoflurophate,DFP|Diisopropoxyphosphoryl Fluoride|Diisopropy...,"[isoflurophate, dfp, diisopropoxyphosphoryl fl..."
3,6-thioguanine,"TG|2-Amino 6MP|2-Amino-1,7-dihydro-6H-purine-6...","[6-thioguanine, tg, 2-amino 6mp, 2-amino-1,7-d..."
4,melatonin,{N-[2-(5-methoxy-1H-indol-3-yl)ethyl]-} Acetam...,"[melatonin, {n-[2-(5-methoxy-1h-indol-3-yl)eth..."
...,...,...,...
2381,Somatrem,Protropin|Somatrem (genetical recombination),"[somatrem, protropin, somatrem (genetical reco..."
2382,spiraprilat,SCH-33861,"[spiraprilat, sch-33861]"
2383,stepronin,Stepronin|Stepronine|Stepronino|Steproninum|Ti...,"[stepronin, stepronin, stepronine, stepronino,..."
2384,sulfacytine,1-ethyl N4-sulfanilylcytosin|1-ethyl-N-sulfani...,"[sulfacytine, 1-ethyl n4-sulfanilylcytosin, 1-..."


In [6]:
df3 = pd.read_json(r"code/clinical_trials_2015.jsonl", lines = True)
df3_drugs = df3[df3['intervention_type']== "Drug"]
"""I filtered by intervention type drug here I was not 
expecting to find matches in the other categories. 
This can easily be changed with the following olumn if required."""
#df3_drugs = df3
df3_drugs['intervention_name'] = df3_drugs['intervention_name'].str.lower()
df3_drugs.sample(n=10)

,nct_id,intervention_name,intervention_type
19096,NCT02614560,melphalan,Drug
21656,NCT02696330,placebo,Drug
20122,NCT02636842,aripiprazole lauroxil,Drug
22894,NCT02780011,brentuximab vedotin,Drug
25500,NCT03262636,indocyanine green,Drug
147,NCT01675154,slx-4090 placebo,Drug
8646,NCT02433236,fostamatinib disodium tablet 150 mg,Drug
13483,NCT02513693,standard neuromuscular blockade,Drug
7780,NCT02419001,period 1 - treatment sequence ab,Drug
12907,NCT02503605,urinary fsh,Drug


In [9]:
words_to_remove = ["and", "intravenous", "+", "- ", "/", r"\(.*\)", "®", " or"]
df3_drugs['matches'] = df3_drugs['intervention_name']

for item in words_to_remove:
    df3_drugs['matches'] = df3_drugs['matches'].str.replace(item,",")

df3_drugs['matches'] = df3_drugs['matches'].str.lower().str.split(',')
#df3_drugs['matches']  = df3_drugs['matches'].str.replace(r"\(.*\)","")
df3_drugs.sample(n=5)#.intervention_type.sample(n=1).reset_index()

,nct_id,intervention_name,intervention_type,matches
2714,NCT02316249,estradiol,Drug,[estradiol]
15899,NCT02556086,sofosbuvir,Drug,[sofosbuvir]
791,NCT02117817,nabpaclitaxel,Drug,[nabpaclitaxel]
23622,NCT02842931,r-da-epoch-21 + auto-sct,Drug,"[r-da-epoch-21 , auto-sct]"
14952,NCT02538367,yh12852 dr1 4mg,Drug,[yh12852 dr1 4mg]


In [10]:
df3_drugs.sample(n=15)

,nct_id,intervention_name,intervention_type,matches
10766,NCT02469298,placebo to match gsk1325756,Drug,[placebo to match gsk1325756]
7566,NCT02415179,inhaled fluticasone/salmeterol,Drug,"[inhaled fluticasone, salmeterol]"
14877,NCT02537171,apatinib,Drug,[apatinib]
14351,NCT02528513,propofol,Drug,[propofol]
6685,NCT02400749,placebo,Drug,[placebo]
22166,NCT02726828,intrathecal ketamine,Drug,[intrathecal ketamine]
4142,NCT02352558,bbi608,Drug,[bbi608]
18797,NCT02608996,placebo,Drug,[placebo]
19093,NCT02614547,placebo,Drug,[placebo]
4102,NCT02351856,"arry-371797, p38 inhibitor, oral",Drug,"[arry-371797, p38 inhibitor, , al]"


In [16]:
drugs_list = list(df2['alternatives'])
#print(drugs_list)

def match_drugs(matches):
    """iterates over the full list of drugs available in drugs.csv, and
    cleaned using regex and string replace methods. Will return the first
    value of the list, as this is the value from 'itemLabel' in drugs.csv """
    
    #print(drugs_list)
    match = ""
    for item in matches:
        item = item.strip()
        for sublist in drugs_list:
            if item not in sublist:
                continue
            else:
                #I just put that in there, to makes multiple matches more readable
                match += " "
                match += sublist[0]
                continue
    return match
            
             
df3_drugs['drugs'] = df3_drugs['matches'].apply(match_drugs)
df3_drugs
#can see on index 25644 that hyaluronan was matched to hyaluronic acid

,nct_id,intervention_name,intervention_type,matches,drugs
0,NCT00293735,labetalol (seizure prevention),Drug,"[labetalol , ]",labetalol olmesartan
1,NCT00293735,mgso4 (seizure prevention),Drug,"[mgso4 , ]",magnesium sulfate olmesartan
9,NCT00749853,follicle stimulating hormone,Drug,[follicle stimulating hormone],urofollitropin
16,NCT01003028,remifentanil (control) - target controlled inf...,Drug,"[remifentanil , , target controlled infusion ...",remifentanil olmesartan
17,NCT01003028,remifentanil (limited) - target controlled inf...,Drug,"[remifentanil , , target controlled infusion ...",remifentanil olmesartan
...,...,...,...,...,...
25631,NCT03692299,metronidazole,Drug,[metronidazole],metronidazole
25638,NCT03705962,tobramycin,Drug,[tobramycin],tobramycin
25644,NCT03800810,hyaluronic acid,Drug,[hyaluronic acid],hyaluronan
25649,NCT03825939,intravenous placebo,Drug,"[, placebo]",olmesartan


In [17]:
df3_drugs = df3_drugs[df3_drugs['drugs'] != ""]
df3_drugs[df3_drugs['nct_id'] == 'NCT01969578']

,nct_id,intervention_name,intervention_type,matches,drugs
402,NCT01969578,bicalutamide + triptorelin,Drug,"[bicalutamide , triptorelin]",bicalutamide triptorelin
403,NCT01969578,carboplatin + paclitaxel,Drug,"[carboplatin , paclitaxel]",carboplatin paclitaxel
404,NCT01969578,cisplatin + doxorubicin,Drug,"[cisplatin , doxorubicin]",cisplatin doxorubicin


In [35]:
df_output = df3_drugs[['nct_id', 'drugs']]


#df_output = df_output.to_json()


In [20]:
json_output = df_output.to_json(orient="records")

parsed = json.loads(json_output)
json.dumps(parsed, indent=0) 
#with open("task1b.json", "w") as outfile: 
#    json.dump(json_output, outfile)

'[\n{\n"nct_id": "NCT00293735",\n"drugs": " labetalol olmesartan"\n},\n{\n"nct_id": "NCT00293735",\n"drugs": " magnesium sulfate olmesartan"\n},\n{\n"nct_id": "NCT00749853",\n"drugs": " urofollitropin"\n},\n{\n"nct_id": "NCT01003028",\n"drugs": " remifentanil olmesartan"\n},\n{\n"nct_id": "NCT01003028",\n"drugs": " remifentanil olmesartan"\n},\n{\n"nct_id": "NCT01112878",\n"drugs": " clonidine"\n},\n{\n"nct_id": "NCT01112878",\n"drugs": " gabapentin"\n},\n{\n"nct_id": "NCT01123486",\n"drugs": " hydromorphone"\n},\n{\n"nct_id": "NCT01132482",\n"drugs": " sildenafil"\n},\n{\n"nct_id": "NCT01171482",\n"drugs": " fluorouracil"\n},\n{\n"nct_id": "NCT01171482",\n"drugs": " mitomycin"\n},\n{\n"nct_id": "NCT01171482",\n"drugs": " sorafenib"\n},\n{\n"nct_id": "NCT01232647",\n"drugs": " olmesartan"\n},\n{\n"nct_id": "NCT01232647",\n"drugs": " (e)-phytonadione"\n},\n{\n"nct_id": "NCT01255046",\n"drugs": " donepezil olmesartan"\n},\n{\n"nct_id": "NCT01333267",\n"drugs": " preotact olmesartan"\n},\

In [21]:
output_dict_nct = dict(zip(df_output.nct_id, df_output.drugs))
with open("task1.json", "w") as outfile: 
    json.dump(output_dict_nct, outfile)
output_dict_nct

{'NCT00293735': ' magnesium sulfate olmesartan',
 'NCT00749853': ' urofollitropin',
 'NCT01003028': ' remifentanil olmesartan',
 'NCT01112878': ' gabapentin',
 'NCT01123486': ' hydromorphone',
 'NCT01132482': ' sildenafil',
 'NCT01171482': ' sorafenib',
 'NCT01232647': ' (e)-phytonadione',
 'NCT01255046': ' donepezil olmesartan',
 'NCT01333267': ' preotact olmesartan',
 'NCT01360034': ' nifedipine',
 'NCT01408849': ' omeprazole',
 'NCT01445691': ' aminolevulinic acid olmesartan',
 'NCT01485393': ' zolpidem',
 'NCT01528618': ' gemcitabine cisplatin',
 'NCT01528865': ' lamivudine',
 'NCT01530984': ' ipilimumab',
 'NCT01561222': ' calcitriol',
 'NCT01589497': ' rifampicin',
 'NCT01613677': ' buparlisib',
 'NCT01614197': ' temsirolimus',
 'NCT01663558': ' imiquimod',
 'NCT01675154': ' orlistat',
 'NCT01701349': ' paclitaxel carboplatin',
 'NCT01718444': ' progesterone',
 'NCT01733043': ' dexmedetomidine',
 'NCT01739556': ' olmesartan',
 'NCT01762644': ' olmesartan',
 'NCT01769911': ' melph

# Task 2:
    

This matches the drugs found in the trials to the descriptions from usan_stems

In [22]:
df_usan_all = pd.read_csv(r"code/usan_stems.csv", header=None, sep='\n')
df_usan_all = df_usan_all[0].str.split(',', expand=True)
headers = list(df_usan_all.iloc[0].fillna("rename")) 
headers = [i.replace('\"', "") for i in headers]
print(headers)
#temporary renaming empty the empty columns
df_usan_all.columns = ['name', 'stem', 'definition', 'example', '1', '2', '3', '4', '5', '6']
df_usan_all.drop(index=0, inplace=True)
df_usan_all

['name', 'stem', 'definition', 'example', '', 'rename', 'rename', 'rename', 'rename', 'rename']


,name,stem,definition,example,1,2,3,4,5,6
1,abine,"""-abine""","""nucleoside antiviral or antineoplastic agents","cytarabine or azarabine derivatives""",,,None,None,None,None
2,ac,"""-ac""","""anti-inflammatory agents (acetic acid derivat...","""bromfenac""",,None,None,None,None,None
3,subgroup:,,,,,None,None,None,None,None
4,,"""-zolac""","""anti-inflammatory - pyrazole acetic acid deri...",""" pyrazole acetic acid derivatives""","""rovazolac""",None,None,None,None,None
5,acetam,"""-acetam""","""nootropic agents (learning",cognitive enhancers),"piracetam type""",,,None,None,None
...,...,...,...,...,...,...,...,...,...,...
758,zolid,"""-zolid""","""oxazolidinone antibacterials""","""eperezolid""",,None,None,None,None,None
759,zomib,"""-zomib""","""proteozome inhibitors""","""bortezomib""",,None,None,None,None,None
760,zotan,"""-zotan""","""5-HT1A receptor agonists/antagonists acting p...","""robalzotan""",,None,None,None,None,None
761,zotide,"""-zotide""","""peptides""",,,None,None,None,None,None


In [23]:

df_usan = df_usan_all.loc[~df_usan_all['name'].str.contains("subgroup:",regex=False)]
df_usan = df_usan.loc[~df_usan['name'].str.contains("subgroups:",regex=False)]
df_usan['name'] = df_usan['name'].replace("", np.NaN )
df_usan['name'] = df_usan['name'].fillna(method='ffill')
df_usan['definition'] = df_usan['definition'].str.strip(' " " ')
df_usan = df_usan.fillna("")
df_usan['example_all'] = df_usan['definition'] + "," + df_usan['example'] + ","  + df_usan['1'] + "," + df_usan['2'] + "," + df_usan['3'] + "," + df_usan['4'] + "," + df_usan['5'] + "," + df_usan['6'] 
df_usan['example_all'] = df_usan['example_all'].replace('\"', '')
df_usan.drop(columns = ['example', '1', '2', '3', '4', '5', '6'], inplace = True)
df_usan.sample(n=10)

,name,stem,definition,example_all
415,nixin,"""-nixin""",anti-inflammatory agents (anilinonicotinic aci...,anti-inflammatory agents (anilinonicotinic aci...
342,lanstat,"""-lanstat""",enzyme inhibitors,"enzyme inhibitors,,,,,,,"
96,caleret,"""-caleret""",calcium receptor antagonists,"calcium receptor antagonists,""ronacaleret"",,,,,,"
563,setron,"""-setron""",serotonin 5-HT3 receptor antagonists,"serotonin 5-HT3 receptor antagonists,""ondanset..."
170,dil,"""-dil-",-dil,"-dil,""vasodilators (undefined group)"",""fostedi..."
42,arit,"""-arit""",antirheumatics (lobenzarit type),"antirheumatics (lobenzarit type),""lobenzarit, ..."
606,ster,"""-ster-""",steroids (androgens,"steroids (androgens, anabolics)"",""norgesterone..."
698,tril/trilat,"""-tril",-trilat,"-trilat,""endopeptidase inhibitors"",""candoxatri..."
348,leuton,"""-leuton""",5-lipoxygenase inhibitors,"5-lipoxygenase inhibitors,""setileuton"",,,,,,"
680,tirelin,"""-tirelin""",prehormones or hormone-release stimulating pep...,prehormones or hormone-release stimulating pep...


In [24]:
#dict(df_usan[['name', 'definition']])
usan_dict = dict(zip(df_usan.name, df_usan.definition))
#print(usan_dict)
output_list_usan = []

for drug in list(df3_drugs['drugs']):
    #print("the drug: ", drug)
    drug_usan = {}
    usan_codes = []
    drug_usan['drug'] = drug
    for k, v in usan_dict.items(): 
        if drug.endswith(k): 
            usandict = {}
            #print("k ", k)
            usandict['description'] = v
            usan_codes.append(usandict)
            continue
            #print(usan_codes)
    if len(usan_codes) > 0:
        drug_usan['usan_codes'] = usan_codes
        output_list_usan.append(drug_usan)
            
#drugs_usan    
#print(list(df3_drugs['drugs']))
output_list_usan

[{'drug': ' labetalol olmesartan',
  'usan_codes': [{'description': 'angiotensin II receptor antagonists'}]},
 {'drug': ' magnesium sulfate olmesartan',
  'usan_codes': [{'description': 'angiotensin II receptor antagonists'}]},
 {'drug': ' remifentanil olmesartan',
  'usan_codes': [{'description': 'angiotensin II receptor antagonists'}]},
 {'drug': ' remifentanil olmesartan',
  'usan_codes': [{'description': 'angiotensin II receptor antagonists'}]},
 {'drug': ' sildenafil',
  'usan_codes': [{'description': '(PDE5) inhibitors containing a sulfonamide moiety'}]},
 {'drug': ' fluorouracil',
  'usan_codes': [{'description': 'uracil type antineoplastics'},
   {'description': 'uracil derivatives used as thyroid antagonists and as antineoplastics'}]},
 {'drug': ' mitomycin',
  'usan_codes': [{'description': 'antibiotics (Streptomyces strain)'}]},
 {'drug': ' sorafenib',
  'usan_codes': [{'description': 'rapidly accelerated fibrosarcoma (RAF) kinase inhibitors'}]},
 {'drug': ' olmesartan',
  '

# Task 3:

In [26]:
def task_3(output_list_usan, output_dict_nct):
    output_list_3 = []
    for drug in output_list_usan:
        drug_description = {}
        trials = []
        #print(drug['drug'])
        #print(drug["usan_codes"][0]["description"])
        for k,v in output_dict_nct.items():
            #print(k, v)
            if v == drug['drug']:
                #print("YES")                
                trials.append(k)        
        if len(trials) > 0:
            drug_description['description'] = drug["usan_codes"][0]["description"]
            drug_description['trials'] = trials
            output_list_3.append(drug_description)
    return output_list_3
                

In [27]:
task_3(output_list_usan, output_dict_nct)

[{'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT00293735', 'NCT02496676']},
 {'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT01003028', 'NCT02481791']},
 {'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT01003028', 'NCT02481791']},
 {'description': '(PDE5) inhibitors containing a sulfonamide moiety',
  'trials': ['NCT01132482',
   'NCT02277132',
   'NCT02304705',
   'NCT02337881',
   'NCT02364882',
   'NCT02387450',
   'NCT02414204',
   'NCT02524184',
   'NCT02595541',
   'NCT02626182',
   'NCT02795351']},
 {'description': 'uracil type antineoplastics',
  'trials': ['NCT02235324', 'NCT02325986', 'NCT02500940', 'NCT02573220']},
 {'description': 'antibiotics (Streptomyces strain)',
  'trials': ['NCT01953406', 'NCT02106572']},
 {'description': 'rapidly accelerated fibrosarcoma (RAF) kinase inhibitors',
  'trials': ['NCT01171482',
   'NCT02129322',
   'NCT02354898',
   'NCT02358395',
   'NCT02406521',
   'NCT02423239',


# Task 4:

In [28]:
task3_list = task_3(output_list_usan, output_dict_nct)
task3_list

[{'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT00293735', 'NCT02496676']},
 {'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT01003028', 'NCT02481791']},
 {'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT01003028', 'NCT02481791']},
 {'description': '(PDE5) inhibitors containing a sulfonamide moiety',
  'trials': ['NCT01132482',
   'NCT02277132',
   'NCT02304705',
   'NCT02337881',
   'NCT02364882',
   'NCT02387450',
   'NCT02414204',
   'NCT02524184',
   'NCT02595541',
   'NCT02626182',
   'NCT02795351']},
 {'description': 'uracil type antineoplastics',
  'trials': ['NCT02235324', 'NCT02325986', 'NCT02500940', 'NCT02573220']},
 {'description': 'antibiotics (Streptomyces strain)',
  'trials': ['NCT01953406', 'NCT02106572']},
 {'description': 'rapidly accelerated fibrosarcoma (RAF) kinase inhibitors',
  'trials': ['NCT01171482',
   'NCT02129322',
   'NCT02354898',
   'NCT02358395',
   'NCT02406521',
   'NCT02423239',


In [31]:
def make_task4_list():
    trial_counts_list= []

    for i in range(0,len(task3_list)-1,2):
        trial_counts_dict = {}
        trial_counts_dict["description1"] = task3_list[i]["description"]
        trial_counts_dict["description2"] = task3_list[i+1]["description"]
        trial_counts_dict["trial_count"] = len(task3_list[i]["trials"]) + len(task3_list[i+1]["trials"])
        print(trial_counts_dict)
        trial_counts_list.append(trial_counts_dict)
    return trial_counts_list
    
make_task4_list()
    

{'description1': 'angiotensin II receptor antagonists', 'description2': 'angiotensin II receptor antagonists', 'trial_count': 4}
{'description1': 'angiotensin II receptor antagonists', 'description2': '(PDE5) inhibitors containing a sulfonamide moiety', 'trial_count': 13}
{'description1': 'uracil type antineoplastics', 'description2': 'antibiotics (Streptomyces strain)', 'trial_count': 6}
{'description1': 'rapidly accelerated fibrosarcoma (RAF) kinase inhibitors', 'description2': 'angiotensin II receptor antagonists', 'trial_count': 329}
{'description1': 'angiotensin II receptor antagonists', 'description2': 'angiotensin II receptor antagonists', 'trial_count': 2}
{'description1': 'angiotensin II receptor antagonists', 'description2': 'angiotensin II receptor antagonists', 'trial_count': 315}
{'description1': '(nifedipine type)', 'description2': 'proton pump inhibitors', 'trial_count': 13}
{'description1': 'angiotensin II receptor antagonists', 'description2': 'hypnotics/sedatives (zol

[{'description1': 'angiotensin II receptor antagonists',
  'description2': 'angiotensin II receptor antagonists',
  'trial_count': 4},
 {'description1': 'angiotensin II receptor antagonists',
  'description2': '(PDE5) inhibitors containing a sulfonamide moiety',
  'trial_count': 13},
 {'description1': 'uracil type antineoplastics',
  'description2': 'antibiotics (Streptomyces strain)',
  'trial_count': 6},
 {'description1': 'rapidly accelerated fibrosarcoma (RAF) kinase inhibitors',
  'description2': 'angiotensin II receptor antagonists',
  'trial_count': 329},
 {'description1': 'angiotensin II receptor antagonists',
  'description2': 'angiotensin II receptor antagonists',
  'trial_count': 2},
 {'description1': 'angiotensin II receptor antagonists',
  'description2': 'angiotensin II receptor antagonists',
  'trial_count': 315},
 {'description1': '(nifedipine type)',
  'description2': 'proton pump inhibitors',
  'trial_count': 13},
 {'description1': 'angiotensin II receptor antagonists'